File 1: config.py

In [25]:
import pytz
from datetime import time

# --- TRADING PARAMETERS ---
SYMBOL = "EURUSD"       # Trading Pair
DAILY_TIMEFRAME = "D1"  # Daily Candlestick for Bias
ENTRY_TIMEFRAMES = ["H4", "H1"] # Higher Timeframes for FVG search
SL_BUFFER_PIPS = 10     # Buffer added to the SL (in pips)

# --- DAILY OPEN TIME (Switch Here) ---
# Use "MIDNIGHT" for 00:00 NYT open
# Use "ASIAN" for 17:00 NYT (start of Asian session) open
DAILY_OPEN_TIME = "ASIAN" 

# --- TIME ZONE AND KILLER ZONES (NYT) ---
NY_TIMEZONE = pytz.timezone('America/New_York')

KILLER_ZONES = [
    (time(2, 0), time(5, 0)),  # London Killer Zone (LKZ)
    (time(7, 0), time(10, 0)), # New York AM Killer Zone (NY AM KZ)
    (time(13, 0), time(16, 0)) # New York PM Killer Zone (NY PM KZ)
]

# --- LOT SIZING AND RISK ---
RISK_PER_TRADE_PERCENT = 0.01 # 1% Risk if using equity calculation
# Customize lot size per asset (overrides risk calculation if specified)
CUSTOM_LOT_SIZES = {
    "EURUSD": 0.10, 
    "XAUUSD": 0.05, 
}

# --- BROKER/API SETUP ---
MAGIC_NUMBER = 246810  # Unique identifier for this bot's trades
# Placeholder credentials. Use python-dotenv for production security.
ACCOUNT_ID = "YOUR_ACCOUNT_ID"
PASSWORD = "YOUR_PASSWORD"
SERVER = "YOUR_SERVER"

File 2: api_client.py

In [26]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import time

from config import SYMBOL
import settings

class BrokerClient:
    def __init__(self):
        """Initializes the MT5 connection."""
        MT5_USER, MT5_PASS, MT5_SERVER, MT5_PATH, _, _ = settings.synthetic()
        if not mt5.initialize(path=MT5_PATH, login=MT5_USER, password=MT5_PASS, server=MT5_SERVER):
            print(f"MT5 initialization failed: {mt5.last_error()}")
            # Attempt a specific login if needed, otherwise raise exception
            if not mt5.login(MT5_USER, password=MT5_PASS, server=MT5_SERVER):
                print(f"MT5 login failed: {mt5.last_error()}")
                raise Exception("Broker connection failed.")
        
        # Get symbol info once for required constants
        self.symbol_info = mt5.symbol_info(SYMBOL)
        if not self.symbol_info:
             raise Exception(f"Failed to get symbol info for {SYMBOL}.")

    def get_data(self, symbol, timeframe_str, count):
        """Fetches OHLC data and returns a DataFrame."""
        
        # Map string timeframe to MT5 constant
        tf_map = {"D1": mt5.TIMEFRAME_D1, "H4": mt5.TIMEFRAME_H4, "H1": mt5.TIMEFRAME_H1, 
                  "M15": mt5.TIMEFRAME_M15, "M5": mt5.TIMEFRAME_M5}
        
        timeframe = tf_map.get(timeframe_str)
        if not timeframe:
            raise ValueError(f"Invalid timeframe string: {timeframe_str}")

        rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, count)
        if rates is None:
            print(f"Error getting rates for {symbol} {timeframe_str}: {mt5.last_error()}")
            return pd.DataFrame()

        df = pd.DataFrame(rates)
        df['time'] = pd.to_datetime(df['time'], unit='s')
        df.set_index('time', inplace=True)
        return df[['open', 'high', 'low', 'close', 'tick_volume']].rename(
            columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close'}
        )
    
    def get_min_lot_size(self, symbol):
        """Gets the minimum allowed volume for the asset."""
        return self.symbol_info.min_volume

    def get_min_stop_distance(self, symbol):
        """Gets the minimum stop distance (in broker points) from the current price."""
        # Note: 'trade_tick_value' and 'point' are crucial for calculation
        return self.symbol_info.trade_stops_level
    
    def point_to_pip_factor(self, symbol):
        """Converts broker points to pips (e.g., 10 for 5-digit broker)."""
        # A standard pip is 0.0001 (4th decimal). Point is the smallest price change.
        return 10**self.symbol_info.digits / 10**4 # Example for 5-digit pairs

    def calculate_volume(self, symbol, sl_pips):
        """Calculates volume based on risk percent or returns custom/min volume."""
        import config
        
        # 1. Check Custom Lot Size
        if symbol in config.CUSTOM_LOT_SIZES:
            return config.CUSTOM_LOT_SIZES[symbol]

        # 2. Risk Calculation (Simplified for structure)
        # Needs actual price/account balance from MT5
        
        # Placeholder Risk Calculation:
        account_info = mt5.account_info()
        if account_info is None:
            print("Failed to get account info.")
            return self.get_min_lot_size(symbol)
            
        equity = account_info.equity
        risk_amount = equity * config.RISK_PER_TRADE_PERCENT
        
        # Highly specific calculation required here (based on trade lot size, contract size, currency, etc.)
        # ... Volume calculation logic ...
        calculated_volume = self.get_min_lot_size(symbol) * 5 # Example calculation 
        
        # 3. Check and Use Minimum Lot Size
        min_lot = self.get_min_lot_size(symbol)
        return max(calculated_volume, min_lot)

    def adjust_price(self, symbol, price_level, is_sl=True):
        """Adjusts SL/TP price to meet broker's minimum distance."""
        
        min_distance_points = self.get_min_stop_distance(symbol)
        point = self.symbol_info.point

        # Get the current BID/ASK for reference
        current_tick = mt5.symbol_info_tick(symbol)
        if current_tick is None: return price_level # Return original if data fails

        # Check for proximity to market (simple check: distance > min_distance)
        distance = abs(price_level - current_tick.last) 
        
        if distance < min_distance_points * point:
            # Adjust outwards
            adjustment = (min_distance_points + 5) * point # Add small buffer (5 points)
            
            if is_sl:
                # Move SL further away from the current price
                if price_level > current_tick.last: # SL is above market (Short)
                    return price_level + adjustment
                else: # SL is below market (Long)
                    return price_level - adjustment
            # Add TP logic if necessary
        
        return price_level

    def place_order(self, symbol, order_type, price, sl, tp, volume):
        """Executes the trade."""
        
        if order_type == 'BUY':
            action = mt5.TRADE_ACTION_BUY
            order_type_mt5 = mt5.ORDER_TYPE_BUY_LIMIT
        else: # SELL
            action = mt5.TRADE_ACTION_SELL
            order_type_mt5 = mt5.ORDER_TYPE_SELL_LIMIT

        request = {
            "action": action,
            "symbol": symbol,
            "volume": volume,
            "type": order_type_mt5,
            "price": price,
            "sl": sl,
            "tp": tp,
            "deviation": 20, # Acceptable deviation in points
            "magic":246810,
            "comment": "ICT_Bias_Bot",
            "type_time": mt5.ORDER_TIME_GTC, # Good Till Cancelled
            "type_filling": mt5.ORDER_FILLING_FOK, # Fill Or Kill
        }

        result = mt5.order_send(request)
        print(f"Order sent. Result: {result}")
        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"Order failed: {mt5.last_error()}")

    def check_and_manage_positions(self):
        """Monitors open positions and applies trade management rules (e.g., FVG reversal exit)."""
        # --- TO BE IMPLEMENTED: Check if any open position's LTF entry FVG/OB/BB has been violated
        # Requires tracking the specific LTF zone price used for entry.
        pass

File 3: strategy.py

In [27]:
import pandas as pd
import numpy
from datetime import datetime, time
import pytz

import config # Import configuration

def determine_daily_bias(daily_data):
    """Determines the daily bias (Long/Short/No Trade) based on previous day's close vs PDH/PDL."""
    
    prev_day = daily_data.iloc[-2] # Previous day's candle
    pdh = prev_day['High']
    pdl = prev_day['Low']
    
    current_close = daily_data.iloc[-1]['Close']
    current_high = daily_data.iloc[-1]['High']
    current_low = daily_data.iloc[-1]['Low']

    # 1. Bullish Continuation (Body closes above PDH)
    if current_close >= pdh:
        return 'LONG_CONTINUATION'
    
    # 2. Bearish Reversal (Sweep PDH, Close below)
    if current_high >= pdh and current_close < pdh:
        return 'SHORT_REVERSAL'
        
    # 3. Bullish Reversal (Sweep PDL, Close above)
    if current_low <= pdl and current_close > pdl:
        return 'LONG_REVERSAL'

    # 4. Indecision (Range Inside or Sweep Both)
    if (current_high >= pdh and current_low <= pdl) or \
       (current_high < pdh and current_low > pdl):
        return 'NO_TRADE_CONSOLIDATION'
        
    return 'NO_TRADE_OTHER'

def is_in_killer_zone(dt_obj):
    """Checks if a datetime object (localized to NYT) falls within a Killer Zone."""
    
    # Ensure the datetime object is localized to NYT
    if dt_obj.tzinfo is None or dt_obj.tzinfo.utcoffset(dt_obj) is None:
        dt_obj = config.NY_TIMEZONE.localize(dt_obj)

    ny_time = dt_obj.astimezone(config.NY_TIMEZONE).time()
    
    for start_time, end_time in config.KILLER_ZONES:
        if start_time <= ny_time <= end_time:
            return True
    return False

def check_fvg(candles):
    """
    Identifies the most recent valid FVG (3-candle pattern).
    Returns (FVG_Start_Price, FVG_End_Price, FVG_Type) or None.
    """
    # --- TO BE IMPLEMENTED: Robust FVG detection logic ---
    # Simplified placeholder for structure:
    if len(candles) < 3:
        return None
        
    c1, c2, c3 = candles.iloc[-3], candles.iloc[-2], candles.iloc[-1]
    
    # Bullish FVG (c1.High < c3.Low)
    if c1['High'] < c3['Low']:
        return (c1['High'], c3['Low'], 'BULLISH')
    
    # Bearish FVG (c1.Low > c3.High)
    if c1['Low'] > c3['High']:
        return (c3['High'], c1['Low'], 'BEARISH')
        
    return None

def check_fvg_location(fvg_info, daily_open_price, daily_bias):
    """Checks if FVG is correctly positioned relative to the Daily Open Price."""
    fvg_start_price, fvg_end_price, fvg_type = fvg_info

    # Bullish logic (FVG must be below or at the daily open price)
    if "LONG" in daily_bias:
        return fvg_end_price <= daily_open_price

    # Bearish logic (FVG must be above or at the daily open price)
    elif "SHORT" in daily_bias:
        return fvg_start_price >= daily_open_price
    
    return False

def check_mss_and_pullback_entry(ltf_data, daily_bias, sl_buffer):
    """
    Checks for MSS, waits for pullback to FVG/OB/BB, and calculates SL/TP.
    
    Returns a dictionary with 'entry_price', 'sl_price', 'tp_price' or None.
    """
    
    # 1. Identify MSS (Market Structure Shift) on LTF
    # --- TO BE IMPLEMENTED: Find the swing high/low break ---
    # Placeholder: Check for a recent structural break
    mss_low = ltf_data['Low'].min() # Example: lowest low for long setup
    mss_high = ltf_data['High'].max() # Example: highest high for short setup
    
    # Assuming MSS logic returns the price level that defines the SL point
    if "LONG" in daily_bias:
        sl_definition_level = mss_low # SL goes below this low
    else:
        sl_definition_level = mss_high # SL goes above this high

    # 2. Identify Pullback Zone (FVG/OB/BB) created by the MSS move
    # --- TO BE IMPLEMENTED: Find the best mitigation zone (FVG/OB/BB) ---
    # Placeholder: Using a simple recent candle open/close as entry reference
    entry_price = ltf_data.iloc[-1]['Open'] # Example entry at current candle open

    # 3. Calculate SL
    pip_value = 0.0001 / 10 # Example point value for 5-digit EURUSD
    sl_pips_amount = sl_buffer * pip_value
    
    if "LONG" in daily_bias:
        sl_price = sl_definition_level - sl_pips_amount
    else:
        sl_price = sl_definition_level + sl_pips_amount
        
    # 4. Calculate TP (Requires knowing the previous day's high/low)
    # This data must be fetched or passed in. Assuming previous day data available.
    
    # --- TO BE IMPLEMENTED: Robust TP calculation (R:R, PDH/L, New Extreme) ---
    tp_price = entry_price + abs(entry_price - sl_price) * 2 # Example R:R 1:2
    
    # Check if entry is valid (i.e., if price is currently mitigating the zone)
    is_mitigating = True # Placeholder check: Must verify current price is touching FVG/OB/BB

    if is_mitigating:
        return {
            'entry_price': entry_price, 
            'sl_price': sl_price, 
            'tp_price': tp_price
        }
    
    return None

File 4: main.py

In [28]:
import time
from datetime import datetime
import pytz

from api_client import BrokerClient
import strategy
import config

def get_daily_open_price(client):
    """Fetches the Daily Open Price based on the config setting (0:00 or 17:00 NYT)."""
    
    # Assuming the broker's daily candle open aligns with one of the NYT times
    if config.DAILY_OPEN_TIME == "ASIAN":
        # Get the open price of the last closed daily candle (opened 17:00 NYT)
        daily_data = client.get_data(config.SYMBOL, config.DAILY_TIMEFRAME, 2)
        return daily_data.iloc[-1]['Open']
        
    elif config.DAILY_OPEN_TIME == "MIDNIGHT":
        # Get price at 00:00 NYT (Requires specific logic not standard in MT5 data copy)
        # Placeholder: Fetch H1 data and find the 00:00 NYT candle open
        h1_data = client.get_data(config.SYMBOL, "H1", 30)
        
        for index, row in h1_data.iterrows():
            dt_ny = index.tz_localize('UTC').astimezone(config.NY_TIMEZONE)
            if dt_ny.hour == 0 and dt_ny.minute == 0:
                return row['Open']

    return None # Return None if unable to determine

def main():
    try:
        client = BrokerClient()
        print("Broker connection successful. Starting bot...")
    except Exception as e:
        print(f"FATAL ERROR: Could not initialize broker client. {e}")
        return

    while True:
        try:
            # 1. Daily Bias Check (Performed on the latest full Daily candle data)
            daily_data = client.get_data(config.SYMBOL, config.DAILY_TIMEFRAME, 2)
            if daily_data.empty:
                print("Could not fetch daily data. Retrying...")
                time.sleep(60)
                continue
                
            daily_bias = strategy.determine_daily_bias(daily_data)
            print(f"\n[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Daily Bias: {daily_bias}")

            if 'NO_TRADE' in daily_bias:
                time.sleep(3600) # Wait 1 hour if no trade is planned
                continue
            
            daily_open_price = get_daily_open_price(client)
            if daily_open_price is None:
                 time.sleep(60)
                 continue
                 
            # 2. Check for FVG Entry on Entry Timeframes (H1/H4)
            for tf in config.ENTRY_TIMEFRAMES:
                tf_data = client.get_data(config.SYMBOL, tf, 50)
                if tf_data.empty: continue
                
                fvg_info = strategy.check_fvg(tf_data)
                
                if fvg_info:
                    # Check 2a. FVG Location Rule
                    if not strategy.check_fvg_location(fvg_info, daily_open_price, daily_bias):
                        print(f"FVG on {tf} invalidated: Location rule failed.")
                        continue 

                    # Check 2b. Time Mitigation Check (Assuming the current time is the mitigation time)
                    # NOTE: This check should be based on the last closed candle's open time on the H1/H4 chart.
                    last_candle_time = tf_data.index[-1].to_pydatetime()
                    if not strategy.is_in_killer_zone(last_candle_time):
                        print(f"FVG on {tf} mitigated outside Killer Zone. Skipping.")
                        continue 
                        
                    # 3. LTF MSS Confirmation Check
                    ltf = "M5" if tf == "H1" else "M15"
                    ltf_data = client.get_data(config.SYMBOL, ltf, 100)
                    
                    entry_details = strategy.check_mss_and_pullback_entry(
                        ltf_data, daily_bias, sl_buffer=config.SL_BUFFER_PIPS
                    )
                    
                    if entry_details:
                        print(f"--- VALID ENTRY FOUND on {tf} confirmed by {ltf} MSS ---")
                        
                        # 4. Final Trade Preparation and Adjustment
                        sl_price_adjusted = client.adjust_price(
                            config.SYMBOL, entry_details['sl_price'], is_sl=True
                        )
                        tp_price_adjusted = client.adjust_price(
                            config.SYMBOL, entry_details['tp_price'], is_sl=False
                        )
                        
                        # Calculate volume based on adjusted SL
                        sl_pips = abs(sl_price_adjusted - entry_details['entry_price']) * client.point_to_pip_factor(config.SYMBOL)
                        volume = client.calculate_volume(config.SYMBOL, sl_pips)

                        # 5. Execute Trade
                        order_type = 'BUY' if 'LONG' in daily_bias else 'SELL'
                        
                        client.place_order(
                            config.SYMBOL, 
                            order_type, 
                            entry_details['entry_price'], 
                            sl_price_adjusted, 
                            tp_price_adjusted, 
                            volume
                        )
                        # Break out of the timeframe loop after placing a trade
                        break 
        
            # 6. Manage Open Positions
            client.check_and_manage_positions()
        
        except Exception as e:
            print(f"An error occurred in the main loop: {e}")
            
        # Wait before the next check (e.g., 5 minutes)
        time.sleep(300) 

if __name__ == "__main__":
    main()

MT5 initialization failed: (-2, 'Invalid "login" argument')
MT5 login failed: (-2, 'Invalid 1st unnamed argument')
FATAL ERROR: Could not initialize broker client. Broker connection failed.
